**KDDCup Data Analytics with PySpark DF: A structured case study**<a href="#KDDCup-Data-Analytics-with-PySpark-DF:-A-structured-case-study" class="anchor-link">¶</a>
=====================================================================================================================================================================

### Udemy Course: Best Hands-on Big Data Practices and Use Cases using PySpark<a href="#Udemy-Course:-Best-Hands-on-Big-Data-Practices-and-Use-Cases-using-PySpark" class="anchor-link">¶</a>

### Author: Amin Karami (PhD, FHEA)<a href="#Author:-Amin-Karami-(PhD,-FHEA)" class="anchor-link">¶</a>

##### data source: <http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html><a href="#data-source:-http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html" class="anchor-link">¶</a>

In \[ \]:

    ########## ONLY in Colab ##########
    !pip3 install pyspark
    ########## ONLY in Colab ##########

In \[ \]:

    ########## ONLY in Ubuntu Machine ##########
    # Load Spark engine
    !pip3 install -q findspark
    import findspark
    findspark.init()
    ########## ONLY in Ubuntu Machine ##########

In \[ \]:

    # import SparkSession

    from pyspark.sql import SparkSession
    spark = SparkSession.builder.master("local[*]").getOrCreate()

    spark

In \[ \]:

    # Read and Load Data to Spark
    # Data source: http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html

    df = spark.read.text("kddcup.data.gz")
    df.printSchema()

In \[ \]:

    # Print data
    df.show()

In \[ \]:

    # Split data (if needed)

    from pyspark.sql.functions import split

    split_col = split(df['value'], ',')
    df = df.withColumn('Protocol', split_col.getItem(1)) \
           .withColumn('Service', split_col.getItem(2)) \
           .withColumn('flag', split_col.getItem(3)) \
           .withColumn('src_bytes', split_col.getItem(4)) \
           .withColumn('dst_bytes', split_col.getItem(5)) \
           .withColumn('urgent', split_col.getItem(8)) \
           .withColumn('num_failed_logins', split_col.getItem(10)) \
           .withColumn('root_shell', split_col.getItem(13)) \
           .withColumn('guest_login', split_col.getItem(21)) \
           .withColumn('label', split_col.getItem(41)) \
           .drop('value')

    df.show()

    +--------+-------+----+---------+---------+------+-----------------+----------+-----------+-------+
    |Protocol|Service|flag|src_bytes|dst_bytes|urgent|num_failed_logins|root_shell|guest_login|  label|
    +--------+-------+----+---------+---------+------+-----------------+----------+-----------+-------+
    |     tcp|   http|  SF|      215|    45076|     0|                0|         0|          0|normal.|
    |     tcp|   http|  SF|      162|     4528|     0|                0|         0|          0|normal.|
    |     tcp|   http|  SF|      236|     1228|     0|                0|         0|          0|normal.|
    |     tcp|   http|  SF|      233|     2032|     0|                0|         0|          0|normal.|
    |     tcp|   http|  SF|      239|      486|     0|                0|         0|          0|normal.|
    |     tcp|   http|  SF|      238|     1282|     0|                0|         0|          0|normal.|
    |     tcp|   http|  SF|      235|     1337|     0|                0|         0|          0|normal.|
    |     tcp|   http|  SF|      234|     1364|     0|                0|         0|          0|normal.|
    |     tcp|   http|  SF|      239|     1295|     0|                0|         0|          0|normal.|
    |     tcp|   http|  SF|      181|     5450|     0|                0|         0|          0|normal.|
    |     tcp|   http|  SF|      184|      124|     0|                0|         0|          0|normal.|
    |     tcp|   http|  SF|      185|     9020|     0|                0|         0|          0|normal.|
    |     tcp|   http|  SF|      239|     1295|     0|                0|         0|          0|normal.|
    |     tcp|   http|  SF|      181|     5450|     0|                0|         0|          0|normal.|
    |     tcp|   http|  SF|      236|     1228|     0|                0|         0|          0|normal.|
    |     tcp|   http|  SF|      233|     2032|     0|                0|         0|          0|normal.|
    |     tcp|   http|  SF|      238|     1282|     0|                0|         0|          0|normal.|
    |     tcp|   http|  SF|      235|     1337|     0|                0|         0|          0|normal.|
    |     tcp|   http|  SF|      234|     1364|     0|                0|         0|          0|normal.|
    |     tcp|   http|  SF|      239|      486|     0|                0|         0|          0|normal.|
    +--------+-------+----+---------+---------+------+-----------------+----------+-----------+-------+
    only showing top 20 rows

In \[ \]:

    # Increase the number of partitions (if needed) and Build a Temp table

    df = df.repartition(10)
    print(df.rdd.getNumPartitions())

    df.createOrReplaceTempView("df_KDDCup")

    10

Question 1: Count the number of connections for each label<a href="#Question-1:-Count-the-number-of-connections-for-each-label" class="anchor-link">¶</a>
---------------------------------------------------------------------------------------------------------------------------------------------------------

In \[ \]:

    df.groupBy('label').count().orderBy('count', ascending= False).show()

Question 2: Get the list of `Protocols`that are `normal` and `vulnerable to attacks`, where there is NOT `guest login` to the destination addresses.<a href="#Question-2:--Get-the-list-of-Protocolsthat-are-normal-and-vulnerable-to-attacks,-where-there-is-NOT-guest-login-to-the-destination-addresses." class="anchor-link">¶</a>
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In \[ \]:

    sql_query = """ SELECT Protocol,
                    CASE label
                      WHEN 'normal.' THEN 'no attack'
                      ELSE 'attack'
                    END AS State,
                    COUNT(*) as freq
                  FROM df_KDDCup
                  WHERE guest_login != 1
                  GROUP BY Protocol, State
                  ORDER BY Protocol DESC
                """

    spark.sql(sql_query).show()

    +--------+---------+-------+
    |Protocol|    State|   freq|
    +--------+---------+-------+
    |     udp|no attack| 191348|
    |     udp|   attack|   2940|
    |     tcp|no attack| 764894|
    |     tcp|   attack|1101613|
    |    icmp|   attack|2820782|
    |    icmp|no attack|  12763|
    +--------+---------+-------+

Question 3: Apply Some Descriptive Statistics on Numerical Data<a href="#Question-3:-Apply-Some-Descriptive-Statistics-on-Numerical-Data" class="anchor-link">¶</a>
-------------------------------------------------------------------------------------------------------------------------------------------------------------------

In \[ \]:

    # PySpark provides built-in standard Aggregate functions defines in DataFrame API
    from pyspark.sql.functions import *

    summary = df.select(mean(df.src_bytes).alias('AVG(src_byte)'),
                        stddev(df.src_bytes).alias('STD(src_byte)'),
                        min(df.src_bytes).alias('MIN(src_byte)'),
                        max(df.src_bytes).alias('MAX(src_byte)'),
                        last(df.src_bytes).alias('LAST(src_byte)'),
                        skewness(df.src_bytes).alias('SKEW(src_byte)'),
                      )
    summary.show()

    +------------------+-----------------+-------------+-------------+--------------+------------------+
    |     AVG(src_byte)|    STD(src_byte)|MIN(src_byte)|MAX(src_byte)|LAST(src_byte)|    SKEW(src_byte)|
    +------------------+-----------------+-------------+-------------+--------------+------------------+
    |1834.6211752293746|941431.0744911298|            0|          999|          2341|1188.9519100465736|
    +------------------+-----------------+-------------+-------------+--------------+------------------+

In \[ \]:

    groups = df.groupBy("Protocol")
    groups.agg({'src_bytes':'mean', 'dst_bytes':'stddev'}).show()

    +--------+-----------------+------------------+
    |Protocol|   avg(src_bytes)| stddev(dst_bytes)|
    +--------+-----------------+------------------+
    |     tcp|3388.569965326596|1043771.3100418178|
    |     udp|97.22772893848308| 55.43318653434132|
    |    icmp|927.8916893855577|               0.0|
    +--------+-----------------+------------------+

Question 4: A descriptive stats based on `Protocols` and `Labels`<a href="#Question-4:-A-descriptive-stats-based-on-Protocols-and-Labels" class="anchor-link">¶</a>
-------------------------------------------------------------------------------------------------------------------------------------------------------------------

In \[ \]:

    sql_query = """
                               SELECT protocol,
                                 CASE label
                                   WHEN 'normal.' THEN 'no attack'
                                   ELSE 'attack'
                                 END AS state,
                                 COUNT(*) as total_freq,
                                ROUND(AVG(src_bytes), 2) as mean_src_bytes,
                                ROUND(AVG(dst_bytes), 2) as mean_dst_bytes,
                                SUM(urgent) as sum_urgent,
                                SUM(num_failed_logins) as sum_failed_logins,
                                SUM(root_shell) as sum_root_shell,
                                SUM(guest_login) as sum_guest_login
                               FROM df_KDDCup
                               GROUP BY protocol, state
                               ORDER BY 3 DESC
                               """
    spark.sql(sql_query).show()

    +--------+---------+----------+--------------+--------------+----------+-----------------+--------------+---------------+
    |protocol|    state|total_freq|mean_src_bytes|mean_dst_bytes|sum_urgent|sum_failed_logins|sum_root_shell|sum_guest_login|
    +--------+---------+----------+--------------+--------------+----------+-----------------+--------------+---------------+
    |    icmp|   attack|   2820782|        931.68|           0.0|       0.0|              0.0|           0.0|            0.0|
    |     tcp|   attack|   1101928|       4465.81|       2005.96|       4.0|             61.0|          32.0|          315.0|
    |     tcp|no attack|    768670|       1844.29|       4071.32|      35.0|             96.0|         302.0|         3776.0|
    |     udp|no attack|    191348|         98.32|         89.41|       0.0|              0.0|           0.0|            0.0|
    |    icmp|no attack|     12763|         90.68|           0.0|       0.0|              0.0|           0.0|            0.0|
    |     udp|   attack|      2940|          26.4|          0.82|       0.0|              0.0|           0.0|            0.0|
    +--------+---------+----------+--------------+--------------+----------+-----------------+--------------+---------------+

Question 5: Get the frquency of `services` for the original `UDP and ICMP` based `attacks`<a href="#Question-5:-Get-the-frquency-of-services-for-the-original-UDP-and-ICMP-based-attacks" class="anchor-link">¶</a>
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

(hint 1: original attacks: `[dos, u2r, r2l, and probe]`)

(hint 2: returns the `services` and `protocols` center justified)

In \[ \]:

    from pyspark.sql.types import StringType

    def Attack_Category(item):
      if item.replace(".", "") in ['back', 'land', 'neptune', 'pod', 'smurf', 'teardrop']:
        return "DoS"
      elif item.replace(".", "") in ['buffer_overflow', 'loadmodule', 'perl', 'rootkit']:
        return "U2R"
      elif item.replace(".", "") in ['ftp_write', 'guess_passwd', 'multihop', 'phf', 'spy', 'warezclient', 'warezmaster']:
        return "R2L"
      else: return "Probe" 

    def Center_Justify(item):
      return item.center(10)


    spark.udf.register("OrginalAttacks", Attack_Category, StringType())
    spark.udf.register("TextJustify", Center_Justify, StringType())

    sql_query = """
                    SELECT 
                      TextJustify(service) as service,
                      TextJustify(protocol) as protocol,
                      OrginalAttacks(label) as new_label,
                      COUNT(*) as freq
                    FROM df_KDDCup
                    WHERE (protocol = 'udp' OR protocol = 'icmp') AND label != 'normal.'
                    GROUP BY service, new_label, protocol
                    ORDER BY freq DESC
              """

    spark.sql(sql_query).show()

    +---------------+---------------+---------+-------+
    |        service|       protocol|new_label|   freq|
    +---------------+---------------+---------+-------+
    |     ecr_i     |      icmp     |      DoS|2808145|
    |     eco_i     |      icmp     |    Probe|  12570|
    |    private    |      udp      |    Probe|   1688|
    |    private    |      udp      |      DoS|    979|
    |     other     |      udp      |    Probe|    261|
    |     ecr_i     |      icmp     |    Probe|     59|
    |    domain_u   |      udp      |    Probe|      9|
    |     tim_i     |      icmp     |      DoS|      5|
    |     other     |      udp      |      U2R|      3|
    |     urp_i     |      icmp     |    Probe|      3|
    +---------------+---------------+---------+-------+